__Vous décidez de vous fixer de nouveaux challenges ! Vous prenez le livre “Beautiful Stories” de
Shakespeare, plusieurs questions vous viennent naturellement__:
- Quel est le thème qui ressort le plus dans ces belles histoires ? Vous regardez les mots les
plus utilisés afin d’avoir un aperçu du champ lexical de ces histoires.
- Vous avez découvert une superbe histoire que vous voulez partager à un ami, vous vous
lancez dans la découpe des différentes histoires pour les exporter séparément.

In [3]:
import requests
# Étape 1: Télécharger le fichier depuis l'URL
url = 'https://www.gutenberg.org/files/1430/1430-0.txt'
response = requests.get(url) 

# Sauvegarder le contenu dans un fichier local
with open('gutenberg.txt', 'wb') as file:
    file.write(response.content)

In [5]:
# Initialiser Spark
from pyspark import SparkConf, SparkContext 
conf = SparkConf().setAppName("example").setMaster("local")
sc = SparkContext(conf=conf)

# Charger le fichier texte dans un RDD
rdd = sc.textFile("gutenberg.txt")
header = rdd.first()
data_rows = rdd.filter(lambda line: line != header)

24/07/30 16:05:44 WARN Utils: Your hostname, LPYTHON-101 resolves to a loopback address: 127.0.1.1; using 172.22.114.204 instead (on interface eno1)
24/07/30 16:05:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/30 16:05:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/30 16:05:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/07/30 16:06:01 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


# Quel est le thème qui ressort le plus dans ces belles histoires ? 
Vous regardez les mots les plus utilisés afin d’avoir un aperçu du champ lexical de ces histoires.


In [16]:

words = rdd.flatMap(lambda line: line.lower().split())
word_counts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

most_frequent_word = word_counts.sortBy(lambda x: x[1], ascending=False).take(50) 
most_frequent_word

[('the', 2139),
 ('and', 1862),
 ('to', 1415),
 ('.', 1373),
 ('of', 1085),
 ('he', 927),
 ('a', 915),
 ('was', 765),
 ('his', 723),
 ('that', 638),
 ('in', 616),
 ('her', 523),
 ('she', 453),
 ('had', 436),
 ('for', 404),
 ('with', 400),
 ('said', 381),
 ('but', 349),
 ('not', 312),
 ('i', 307),
 ('as', 302),
 ('it', 298),
 ('you', 287),
 ('him', 276),
 ('is', 258),
 ('when', 230),
 ('be', 223),
 ('on', 216),
 ('so', 208),
 ('they', 206),
 ('at', 202),
 ('who', 196),
 ('were', 190),
 ('have', 189),
 ('by', 176),
 ('would', 172),
 ('this', 167),
 ('my', 159),
 ('then', 156),
 ('“i', 155),
 ('all', 152),
 ('from', 150),
 ('--', 142),
 ('will', 141),
 ('your', 138),
 ('king', 127),
 ('one', 120),
 ('their', 117),
 ('said,', 114),
 ('which', 113)]

In [17]:
blacklist = set([
    'the', 'and', 'to', '.', 'of', 'he', 'a', 'was', 'his', 'that', 'in', 'her', 'she', 'had', 'for',
    'with', 'said', 'but', 'not', 'i', 'as', 'it', 'you', 'him', 'is', 'when', 'be', 'on', 'so', 'they',
    'at', 'who', 'were', 'have', 'by', 'would', 'this', 'my', 'then', '“i', 'all', 'from', '--', 'will',
    'your'
])

filtered_words = words.filter(lambda word: word not in blacklist and word.isalpha())
word_counts = filtered_words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)


most_frequent_word = word_counts.sortBy(lambda x: x[1], ascending=False).take(10)
most_frequent_word

[('king', 127),
 ('one', 120),
 ('their', 117),
 ('which', 113),
 ('if', 113),
 ('are', 108),
 ('no', 105),
 ('could', 105),
 ('love', 102),
 ('been', 100)]

In [18]:
print("Les 10 mots les plus fréquents (hors blacklist) :")
for word, count in most_frequent_word:
    print(f"{word}: {count}")

Les 10 mots les plus fréquents (hors blacklist) :
king: 127
one: 120
their: 117
which: 113
if: 113
are: 108
no: 105
could: 105
love: 102
been: 100


# Découper des différentes histoires pour les exporter séparément.